### Housing Prices with Ames Data Set

1. [Imports](#imports)
2. [Data context](#data)
3. [Initial Model Run](#initial)
3. [Exploratory Analysis](#explore)   
4. [Data Cleaning](#clean)
5. [Feature Engineering](#engineer)
6. [Algorithm Selection](#select)
6. [Hyperparameter Tuning](#tune)
7. [Model Training](#train)

#### Imports<a name="imports"></a>

In [1]:
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:.3f}'.format

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set_style("whitegrid")

In [3]:
import my_utilities as mut

In [4]:
# sklearn regressor
from sklearn.pipeline import Pipeline
from sklearn.cross_decomposition import PLSRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.linear_model import LinearRegression, ElasticNet, Lasso, Ridge
from sklearn.metrics import r2_score, mean_squared_error, mean_squared_log_error
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import train_test_split, GridSearchCV

#### Understand Problem<a name="data"></a>
- clean enough to plot 
- preliminary resolution of missing values
- categorize features
- understand target

In [ ]:
file = 'AmesHousing/Data/train.csv'
df = pd.read_csv(file) 
df.columns = [mut.label_uncap_split(col) for col in df.columns]

In [ ]:
#df.head()
#df.dtypes
#df.columns
#df.describe()
#uniq = {col:len(df[col].unique()) for col in df if col in col_obj}
#df.loc[np.where(df.mas_vnr_area.isna())]

In [ ]:
# remove nan for plotting
col_obj = [col for col in df if pd.api.types.is_object_dtype(df[col])]
col_num = [col for col in df if not col in col_obj]
col_nan = [col for col in df if df[col].isna().any()]
col_onn =  set(col_obj) & set(col_nan)
col_fnn =  set(col_num) & set(col_nan)
col_nnn =  set(col_nan) - set(col_onn)
for col in col_obj:
    df[col] = df[col].fillna('NA')
for col in col_fnn:
    df[col] = df[col].fillna(0)

In [ ]:
f, ax = plt.subplots(figsize=(15,6))
sns.distplot(df.sale_price, color="b", kde = False,  ax=ax)
plt.show()

#### initial model run<a name="initial"></a>
Run a simple model to help define our problem.  Maybe that is all that is needed. The results can guide the feature engineering process.  

In [ ]:
# training data and define categorical and numerical data
file = 'AmesHousing/Data/train.csv'
dfn = pd.read_csv(file) 
dfn.columns = [mut.label_uncap_split(col) for col in dfn.columns]
col_cat = [col for col in dfn if pd.api.types.is_object_dtype(dfn[col])]
col_num = [col for col in dfn if not col in col_cat]
col_num.remove('id'); col_num.remove('sale_price')
X = dfn.drop(['id', 'sale_price'], axis=1)
y = dfn['sale_price']

In [ ]:
X_trn, X_tst, y_trn, y_tst = train_test_split( X, y, test_size=0.10, random_state=10)

In [ ]:
# preprocessing
num_tfr = Pipeline(steps=[
    ('sir', SimpleImputer(strategy='constant', fill_value=0)),
    ('ssr', StandardScaler())])

cat_tfr = Pipeline(steps=[
    ('sir', SimpleImputer(strategy='constant', fill_value='NA')),
    ('ohe', OneHotEncoder(handle_unknown='ignore'))])

ppr = ColumnTransformer(
    transformers=[
        ('num', num_tfr, col_num),
        ('cat_tfr', cat_tfr, col_cat)])

est_ols = Pipeline(steps=[('ppr', ppr),
                      ('est', LinearRegression())])

In [ ]:
y_prd = est_ols.fit(X_trn, y_trn).predict(X_tst)
print("R Squared: %.3f" % r2_score(y_prd, y_tst))

In [ ]:
# Extract predicted values.
predicted = y_prd.ravel()
actual = y_tst
# Calculate the error, also called the residual.
residual = actual - predicted
plt.hist(residual)
plt.title('Residual counts');plt.xlabel('Residual');plt.ylabel('Count')
plt.show()

In [ ]:
plt.scatter(predicted, residual)
plt.xlabel('Predicted')
plt.ylabel('Residual')
plt.axhline(y=0)
plt.title('Residual vs. Predicted')
plt.show()

In [ ]:
correlation_matrix = X.corr()
f, ax = plt.subplots(figsize=(18, 15))
sns.heatmap(correlation_matrix, vmax=.8, square=True)
plt.show()

Results were disappointingly good.  

#### Exploratory Analysis

 - Start with Basics

 - Context of Data

 - Plot Numerical Distributions
 
 - Plot Categorical Distributions

 - Plot Segmentations

 - Study Correlations

In [ ]:
file = 'AmesHousing/Data/train.csv'
dfe = pd.read_csv(file) 
dfe.columns = [mut.label_uncap_split(col) for col in dfe.columns]

In [ ]:
col_cat = [col for col in dfe.columns if dfe[col].dtype == 'O']
col_flt = [col for col in dfe.columns if dfe[col].dtype == 'float64']
col_int = [col for col in dfe.columns if dfe[col].dtype == 'int64']
col_nan = [col for col in dfe.columns if dfe[col]a.isna().any()]
col_ord = [col for col in col_int if len(dfe[col].unique()) <= 12]
col_num = list(set(col_int) - set(col_ord))

In [ ]:
plot1 = [col for col in col_obj if (len(df[col].unique()) <= 8)]
plot2 = [col for col in col_obj if (len(df[col].unique()) > 8)]
plot2.remove('neighborhood')

In [ ]:
for col in col_cnn:
    dfe[col] = dfe[col].fillna('NA')

In [ ]:
col_onn =  set(col_obj) & set(col_nan)
col_fnn =  set(col_num) & set(col_nan)
col_nnn =  set(col_nan) - set(col_onn)
for col in col_obj:
    df[col] = df[col].fillna('NA')
for col in col_fnn:
    df[col] = df[col].fillna(0)

In [ ]:
col_onn =  set(col_obj) & set(col_nan)
col_fnn =  set(col_num) & set(col_nan)
col_nnn =  set(col_nan) - set(col_onn)
for col in col_obj:
    df[col] = df[col].fillna('NA')
for col in col_fnn:
    df[col] = df[col].fillna(0)

In [ ]:
cols = 2
rows = len(plot1) // cols 
fig, axs = plt.subplots(rows, cols, sharex='col', sharey='row', figsize=(16, rows * 4))
i = 0
for x in range(rows):
    for y in range(cols):
        sns.countplot(x=plot1[i], data=df, ax=axs[x][y])     
        i += 1
plt.show()

In [ ]:
cols = 2
rows = len(plot1) // cols 
fig, axs = plt.subplots(rows, cols, sharex='col', sharey='row', figsize=(16, rows * 4))
i = 0
for x in range(rows):
    for y in range(cols):
        sns.barplot(x=plot1[i], y="sale_price", data=df, ax=axs[x][y])     
        i += 1
plt.show()

In [ ]:
cols = 1
rows = len(plot2) // cols 
fig, axs = plt.subplots(rows, cols, sharex='col', sharey='row', figsize=(16, rows * 5))

for i in range( rows ):
        sns.barplot(x=plot2[i], y="sale_price", data=df, ax=axs[i])     
plt.show()

In [ ]:
f, ax = plt.subplots(figsize=(16, 24))
sns.barplot(x="sale_price", y='neighborhood', data=df, orient='h', ax=ax)  
plt.show()

In [ ]:
f, ax = plt.subplots(figsize=(15,6))
sns.distplot(df.lot_frontage, color="b", kde = False, norm_hist=True, ax=ax)
plt.show()

In [ ]:
cols = 2
rows = len(col_num) // cols 
fig, axs = plt.subplots(rows, cols, sharex='col', sharey='row', figsize=(16, rows * 5))
i = 0
for x in range(rows):
    for y in range(cols):
        sns.distplot(df[col_num[i]],  ax=axs[x][y])     
        i += 1
plt.show()

#### Data Cleaning<a name="clean"></a>
 - Remove Unwanted observations
 - Fix Structural Errors
 - Filter Unwanted Outliers
 - Handle Missing Data
 

#### Feature Engineering<a name="engineer"></a>

- Transform Target

- Infuse Domain Knowledge

- Create Interaction Features
 
- Combine Sparse Classes

- Add Dummy Variables 

- Remove Unused Features

####  Feature Engineering 1
 - correct feature classifications
 - add interaction feature to account for time related variance
 - rank the ordinal quality and condition values
 - acount for outliers in continuous features

In [83]:
file = 'AmesHousing/Data/train.csv'
dff = pd.read_csv(file) 
dff.columns = [mut.label_uncap_split(col) for col in dff.columns]

In [84]:
# categorize features for preprocessing, correct errors from initial model 
fte_cat = [col for col in dff.columns if dff[col].dtype == 'O']
fte_flt = [col for col in dff.columns if dff[col].dtype == 'float64']
fte_int = [col for col in dff.columns if dff[col].dtype == 'int64']
fte_nan = [col for col in dff.columns if dff[col].isna().any()]
#fte_ord = [col for col in fte_int if len(dff[col].unique()) <= 12]
drop1 = ['id', 'sale_price']

In [85]:
# MSSubClass is categorical feature with integers for values.  change to letters to avoid confusion again
A = dff.sub_class.unique()
dct_scs = {A[i]:chr for (i, chr) in enumerate('ABCDEFGHIJKLMNO')}
dff.sub_class =  mut.fillna_transform_dct(dff.sub_class, dct_scs)
fte_cat.append('sub_class')

In [86]:
dff.garage_yr_blt  = 2016 - dff.garage_yr_blt
dff.year_built     = 2016 - dff.year_built
dff.year_remod_add = 2016 - dff.year_remod_add

In [87]:
# ord_mth for variance across entire time period
dff['ord_mth'] = (dff.yr_sold - 2006) * 12 + dff.mo_sold
fte_tme = ['ord_mth']; drop1.append('yr_sold')

In [88]:
# quality and condition ranks to match 1-10 scale for overall condition 
dct_ord = {'Ex':9,'Gd':7, 'TA':5, 'Fa':3, 'Po':2, 'NA':0}
fte_ord = ['overall_qual','overall_cond']
tfm_ord = ['bsmt_qual', 'bsmt_cond', 'extr_qual', 'extr_cond','heating_qc', 'garage_qual', 'garage_cond','pool_qc' ]

for tfm in tfm_ord:
    dff[tfm] = mut.fillna_transform_dct(dff[tfm], dct_ord)
    fte_ord.append(tfm)  

In [89]:
fte_cat = list(set(fte_cat) - set(fte_ord)) 
fte_cnt = list(set(fte_flt) | set(fte_int) - set(fte_ord) -  set(fte_cat))
fte_ord = list(set(fte_ord) | set(fte_tme))

In [90]:
fte_cat = [fte for fte in fte_cat if not fte in drop1]
fte_cnt = [fte for fte in fte_cnt if not fte in drop1]
fte_ord = [fte for fte in fte_ord if not fte in drop1]

#### Algorithm Selection <a name="select"></a>

In [ ]:
# preprocessing
tfr_cnt = Pipeline(steps=[
    ('sir', SimpleImputer(strategy='constant', fill_value=0)),
    ('ssr', StandardScaler()),
    ('mms', MinMaxScaler())])
tfr_ord = Pipeline(steps=[
    ('sir', SimpleImputer(strategy='constant', fill_value=0)),
    ('mms', MinMaxScaler())])
tfr_cat = Pipeline(steps=[
    ('sir', SimpleImputer(strategy='constant', fill_value='NA')),
    ('ohe', OneHotEncoder(handle_unknown='ignore'))])

ppr = ColumnTransformer(
    transformers=[
        ('cnt', tfr_cnt, fte_cnt),
        ('cat', tfr_cat, fte_cat),
        ('ord', tfr_ord, fte_ord)])

est_ols = Pipeline(steps=[('ppr', ppr), ('est', LinearRegression())])
est_lso = Pipeline(steps=[('ppr', ppr), ('est', Lasso())])
est_rdg = Pipeline(steps=[('ppr', ppr), ('est', Ridge())])
est_ent = Pipeline(steps=[('ppr', ppr), ('est', ElasticNet())])
est_svr = Pipeline(steps=[('ppr', ppr), ('est', SVR())])
est_svr = Pipeline(steps=[('ppr', ppr), ('est', KernelRidge())])
est_gbr = Pipeline(steps=[('ppr', ppr), ('est', GradientBoostingRegressor())])

In [ ]:
X = dff.drop(drop1, axis=1)
y = dff['sale_price']
X_trn, X_tst, y_trn, y_tst = train_test_split( X, y, test_size=0.20)

In [ ]:
ests = [est_ols, est_lso, est_rdg, est_ent, est_svr, est_svr, est_gbr]

In [ ]:
y_prd = est_ols.fit(X_trn, y_trn).predict(X_tst)

In [ ]:
mean_squared_error(y_tst, y_prd)

In [ ]:
print("log mean squared error %.8f" % mean_squared_log_error( y_tst, y_prd))

In [ ]:
prds = [est.fit(X_trn, y_trn).predict(X_tst) for est in ests]


In [ ]:
r_2s = [str(i) + ' : ' + "R Squared: %.3f" % r2_score(y_tst, prd) for (i,prd) in enumerate(prds)]
mses = [str(i) + ' : ' + "mean squared log error: %.3f" % mean_squared_log_error(y_tst, prd)
        for (i,prd) in enumerate(prds)]

In [ ]:
for r_2 in r_2s:
    print(r_2)

In [ ]:
for mse in mses:
    print(mse)

#### Feature Engineering 2
  feature engineering 2
 - combine the sparse classes

In [91]:
drop2 = drop1

In [92]:
dff['bath'] = dff.half_bath * 2 + dff.full_bath * 4 + dff.bsmt_half_bath + dff.bsmt_full_bath
fte_cnt.append('bath'); drop2 = drop2 + ['half_bath', 'full_bath', 'bsmt_half_bath', 'bsmt_full_bath']

In [93]:
# combine condition1 and condition2 into ordinal feature
dct_cdn = {'Artery':-3, 'Feedr':-1, 'Norm':0,'RRNn':-1, 'RRAn':-3, 'PosN':1, 'PosA':3,'RRNe':-1, 'RRAe':-3}
dff.condition1 = dff.condition1.apply(lambda x : dct_cdn[x])
dff.condition2 = dff.condition2.apply(lambda x : dct_cdn[x])
dff['cndn'] = dff.condition1 + dff.condition2
fte_ord.append('cndn'); drop2 = drop2 + ['condition1', 'condition2']

In [94]:
# account for variance in "low_qual_fin_sf" with ratio and delete redundent first and second floor areas
dff['gr_liv_qlty'] = (dff.gr_liv_area - dff.low_qual_fin_sf) / dff.gr_liv_area
fte_cnt.append('gr_liv_qlty'); drop2 = drop2 + ['frst_flr_sf', 'scnd_flr_sf', 'low_qual_fin_sf']

In [95]:
dff['gar_cq'] = dff.garage_cond + dff.garage_qual

In [96]:
fte_ord.append('gr_liv_qlty'); drop2 = drop2 + ['garage_cond', 'garage_qual','garage_area' ]

In [ ]:
       'bsmt_qual', 'bsmt_cond', 'bsmt_exposure', 'bsmt_fin_type1',
       'bsmt_fin_sf1', 'bsmt_fin_type2', 'bsmt_fin_sf2', 'bsmt_unf_sf','total_bsmt_sf'

In [97]:
dff['bmt_cq'] = dff.bsmt_cond + dff.bsmt_qual

In [98]:
dct_bmt = {'GLQ':20, 'ALQ':17, 'Unf':3, 'Rec':11, 'BLQ':14, 'NA':0, 'LwQ':7}
dff.bsmt_fin_type1 =  mut.fillna_transform_dct(dff.bsmt_fin_type1, dct_bmt)
dff.bsmt_fin_type2 =  mut.fillna_transform_dct(dff.bsmt_fin_type2, dct_bmt)

In [101]:
dff['bmt_ttl'] = (dff.bsmt_fin_type1 * dff.bsmt_fin_sf1 + 
                  dff.bsmt_fin_type2 * dff.bsmt_fin_sf2 + dff.bsmt_unf_sf * 3) // 20
                 

In [102]:
fte_ord.append('bmt_cq'); fte_ord.append('bmt_ttl')
drop2 = drop2 + ['bsmt_qual', 'bsmt_cond', 'bsmt_fin_type1', 'bsmt_fin_sf1',
                 'bsmt_fin_type2', 'bsmt_fin_sf2', 'bsmt_unf_sf','total_bsmt_sf']

In [103]:
fte_cat = [fte for fte in fte_cat if not fte in drop2]
fte_cnt = [fte for fte in fte_cnt if not fte in drop2]
fte_ord = [fte for fte in fte_ord if not fte in drop2]

#### Algorithm Selection

#### Hyperparameter Selection<a name="tune"></a>


#### Model Training<a name="train"></a>

In [ ]:
# preprocessing
tfr_cnt = Pipeline(steps=[
    ('sir', SimpleImputer(strategy='constant', fill_value=0)),
    ('ssr', StandardScaler()),
    ('mms', MinMaxScaler())])
tfr_ord = Pipeline(steps=[
    ('sir', SimpleImputer(strategy='constant', fill_value=0)),
    ('mms', MinMaxScaler())])
tfr_cat = Pipeline(steps=[
    ('sir', SimpleImputer(strategy='constant', fill_value='NA')),
    ('ohe', OneHotEncoder(handle_unknown='ignore'))])

ppr = ColumnTransformer(
    transformers=[
        ('cnt', tfr_cnt, fte_cnt),
        ('cat', tfr_cat, fte_cat),
        ('ord', tfr_ord, fte_ord)])

mdl = Pipeline(steps=[('ppr', ppr), ('est', LinearRegression())])

In [ ]:
# use all the data to fit model
mdl = mdl.fit(X,y).predict(X)

#### Model in Production (sorta)

In [ ]:
file = 'AmesHousing/Data/test.csv'
dft = pd.read_csv(file) 
dft.columns = [mut.label_uncap_split(col) for col in dft.columns]
tst_ids = dft.id

In [ ]:
# categorize features for preprocessing, correct errors from initial model 
fte_cat = [col for col in dft.columns if dft[col].dtype == 'O']
fte_flt = [col for col in dft.columns if dft[col].dtype == 'float64']
fte_int = [col for col in dft.columns if dft[col].dtype == 'int64']
fte_nan = [col for col in dft.columns if dft[col].isna().any()]
#fte_ord = [col for col in fte_int if len(dft[col].unique()) <= 12]
drop1 = ['id']
# MSSubClass is categorical feature with integers for values.  change to letters to avoid confusion again
A = dft.sub_class.unique()
dct_scs = {A[i]:chr for (i, chr) in enumerate('ABCDEFGHIJKLMNOP')}
dft.sub_class =  mut.fillna_transform_dct(dft.sub_class, dct_scs)
fte_cat.append('sub_class')
# ord_mth for variance across entire time period
dft['ord_mth'] = (dft.yr_sold - 2006) * 12 + dft.mo_sold
fte_tme = ['ord_mth']; drop1.append('yr_sold')
# quality and condition ranks to match 1-10 scale for overall condition 
dct_ord = {'Ex':9,'Gd':7, 'TA':5, 'Fa':3, 'Po':2, 'NA':0}
fte_ord = ['overall_qual','overall_cond']
tfm_ord = ['bsmt_qual', 'bsmt_cond', 'extr_qual', 'extr_cond','heating_qc', 'garage_qual', 'garage_cond','pool_qc' ]

for tfm in tfm_ord:
    dft[tfm] = mut.fillna_transform_dct(dft[tfm], dct_ord)
    fte_ord.append(tfm)  
fte_cat = list(set(fte_cat) - set(fte_ord)) 
fte_cnt = list(set(fte_flt) | set(fte_int) - set(fte_ord) -  set(fte_cat))
fte_ord = list(set(fte_ord) | set(fte_tme))
fte_cat = [fte for fte in fte_cat if not fte in drop1]
fte_cnt = [fte for fte in fte_cnt if not fte in drop1]
fte_ord = [fte for fte in fte_ord if not fte in drop1]